<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/047_README_Refactored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 📦 Refactoring Our README Agent: Using Tool Decorators

In our original README agent, we manually created and registered each action. While this approach works, it requires us to maintain the tool definitions (functions) separately from their metadata (descriptions and parameters). This separation creates opportunities for these two pieces to become out of sync.

Let’s improve our agent by using **tool decorators** to keep everything together and automatically synchronized.

---

## 🧠 Understanding the Problem

Original manual setup:

```python
def read_project_file(name: str) -> str:
    with open(name, "r") as f:
        return f.read()

# Later, separately, we define metadata about the function
action_registry.register(Action(
    name="read_project_file",
    function=read_project_file,
    description="Reads a file from the project.",
    parameters={
        "type": "object",
        "properties": {
            "name": {"type": "string"}
        },
        "required": ["name"]
    },
    terminal=False
))
```

### ❌ Problem:

* If you change the function’s parameters, you must also update the parameter schema.
* If you change the behavior of the function, you must remember to update the description.
* These manual steps are **error-prone** and may confuse both developers and the LLM.

---

## ✅ The Decorator Solution

Using decorators, we define tools with their metadata inline:

```python
@register_tool(tags=["file_operations", "read"])
def read_project_file(name: str) -> str:
    """Reads and returns the content of a specified project file."""
    with open(name, "r") as f:
        return f.read()

@register_tool(tags=["file_operations", "list"])
def list_project_files() -> List[str]:
    """Lists all Python files in the current project directory."""
    return sorted([file for file in os.listdir(".") if file.endswith(".py")])

@register_tool(tags=["system"], terminal=True)
def terminate(message: str) -> str:
    """Terminates the agent's execution with a final message."""
    return f"{message}\nTerminating..."
```

---

## ⚙️ Refactored `main()` Function

```python
def main():
    goals = [
        Goal(priority=1, name="Gather Information", description="Read each file in the project in order to write a README"),
        Goal(priority=1, name="Terminate", description="Call terminate when done and provide a README in the message")
    ]

    agent = Agent(
        goals=goals,
        agent_language=AgentFunctionCallingActionLanguage(),
        action_registry=PythonActionRegistry(tags=["file_operations", "system"]),
        generate_response=generate_response,
        environment=Environment()
    )

    user_input = "Write a README for this project."
    final_memory = agent.run(user_input)
    print(final_memory.get_memories())

if __name__ == "__main__":
    main()
```

---

## 🔍 Key Improvements

### 1. 📝 Self-Documenting Tools

* Docstrings are used as tool descriptions
* IDEs can show these docs
* Descriptions stay up to date naturally

### 2. 📦 Automatic Parameter Inference

* Function signatures are auto-parsed
* Required parameters are detected automatically
* Reduces schema maintenance

### 3. 🏷️ Logical Organization Through Tags

* `file_operations`, `read`, `system`, etc.
* Makes agent creation filterable and semantic

### 4. 🤖 Simplified Agent Creation

```python
action_registry=PythonActionRegistry(tags=["file_operations", "system"])
```

* No need to manually register tools
* Just filter by relevant tags

### 5. 🔧 Easier Maintenance

* Add tools by decorating functions
* Modify without touching registration code
* Remove tools by removing the decorator

---

## 🧬 Understanding the Flow

1. Decorators register tools globally when the code loads.
2. `ActionRegistry` filters tools by tag.
3. The agent uses the pre-filtered tools to operate.

> This automation reduces errors and makes your codebase easier to manage and scale.





## ✅ Updating a Tool via Decorator Automatically Syncs Metadata

When you use a decorator like `@register_tool`:

```python
@register_tool(tags=["file_operations"])
def read_file(file_path: str) -> str:
    """Reads and returns the content of a file."""
    with open(file_path, 'r') as f:
        return f.read()
```

All the following metadata is automatically derived and stored:

* ✅ **Tool name** (`read_file`)
* ✅ **Docstring becomes the description**
* ✅ **Parameters** inferred from the function signature
* ✅ **Tags** used to organize tools for different agents

---

### 🔄 If You Update the Function Later…

Suppose you update the function’s behavior, signature, or description:

```python
@register_tool(tags=["file_operations", "secure_read"])
def read_file(file_path: str, safe_mode: bool = True) -> str:
    """Securely reads a file with optional safe mode check."""
    if not file_path.endswith(".txt") and safe_mode:
        return "Blocked unsafe file type"
    with open(file_path, 'r') as f:
        return f.read()
```

When the code is run:

* The original `read_file` metadata is **overwritten**.
* The registry automatically stores the **new docstring**, **new signature**, and **updated tags**.
* No manual updates needed elsewhere.

---

## 🔧 Why This Is So Helpful

| Without Decorators                        | With Decorators                       |
| ----------------------------------------- | ------------------------------------- |
| You must update `ActionRegistry` manually | Metadata updates automatically        |
| Risk of mismatch (e.g., wrong schema)     | Guaranteed to match the function      |
| Repetitive code (function + schema)       | Single source of truth (the function) |

---

### ✅ Result: You can confidently modify tools in one place without worrying about forgetting to update the registry.



In [ ]:
# First, we'll define our tools using decorators
@register_tool(tags=["file_operations", "read"])
def read_project_file(name: str) -> str:
    """Reads and returns the content of a specified project file.

    Opens the file in read mode and returns its entire contents as a string.
    Raises FileNotFoundError if the file doesn't exist.

    Args:
        name: The name of the file to read

    Returns:
        The contents of the file as a string
    """
    with open(name, "r") as f:
        return f.read()

@register_tool(tags=["file_operations", "list"])
def list_project_files() -> List[str]:
    """Lists all Python files in the current project directory.

    Scans the current directory and returns a sorted list of all files
    that end with '.py'.

    Returns:
        A sorted list of Python filenames
    """
    return sorted([file for file in os.listdir(".")
                   if file.endswith(".py")])

@register_tool(tags=["system"], terminal=True)
def terminate(message: str) -> str:
    """Terminates the agent's execution with a final message.

    Args:
        message: The final message to return before terminating

    Returns:
        The message with a termination note appended
    """
    return f"{message}\nTerminating..."

def main():
    # Define the agent's goals
    goals = [
        Goal(priority=1,
             name="Gather Information",
             description="Read each file in the project in order to build a deep understanding of the project in order to write a README"),
        Goal(priority=1,
             name="Terminate",
             description="Call terminate when done and provide a complete README for the project in the message parameter")
    ]

    # Create an agent instance with tag-filtered actions
    agent = Agent(
        goals=goals,
        agent_language=AgentFunctionCallingActionLanguage(),
        # The ActionRegistry now automatically loads tools with these tags
        action_registry=PythonActionRegistry(tags=["file_operations", "system"]),
        generate_response=generate_response,
        environment=Environment()
    )

    # Run the agent with user input
    user_input = "Write a README for this project."
    final_memory = agent.run(user_input)
    print(final_memory.get_memories())

if __name__ == "__main__":
    main()



### ✅ 1. Self-Documenting Tools

Each tool now carries its own documentation through **Python docstrings**, and the `@register_tool` decorator extracts this automatically to use as the tool’s **description**.

**Benefits:**

* 📝 **Documentation lives with the function** — reducing duplication and drift.
* 🔁 **Changes to the function = updates to the docs**, automatically reflected in the tool registry.
* 💡 **IDE support** — Docstrings improve auto-complete hints and inline documentation when working in editors.

---

### 🧠 2. Automatic Parameter Inference

The decorator uses Python’s **type hints and function signature** to automatically build the tool’s **parameter schema** (used for OpenAI function calling or internal validation).

**Benefits:**

* 🔧 **No need to manually write schemas** — less boilerplate.
* ✅ **Parameters stay in sync** — changes in the function signature automatically update the schema.
* 📦 **Required parameters** are inferred from the function definition (non-default arguments).

---

### 🏷️ 3. Logical Organization Through Tags

Each tool is now registered with descriptive **tags**, like:

* `"file_operations"` → for file-related utilities
* `"read"`, `"list"` → to describe the specific kind of action
* `"system"` → for agent control or finalization actions (like `terminate`)

**Benefits:**

* 📚 **Easier grouping** — tools can be grouped logically by function or domain.
* 🧩 **Targeted registries** — agents can load only the tools they need by tag.
* 👀 **Improved clarity** — tags communicate purpose clearly to both developers and models.




### ⚙️ 4. Simplified Agent Creation

Agent creation becomes dramatically cleaner and more scalable:

```python
agent = Agent(
    goals=goals,
    agent_language=AgentFunctionCallingActionLanguage(),
    action_registry=ActionRegistry(tags=["file_operations", "system"]),
    generate_response=generate_response,
    environment=Environment()
)
```

**Why it matters:**

* 🧼 No need to manually register each tool
* ✅ Tools are automatically included based on their **tags**
* 🧩 Easy to plug in the right tools for a specific agent's needs

---

### 🛠️ 5. Easier Maintenance

As your project grows, maintaining your agent stays simple and clean:

| Task               | Old Way                                | New Way (with Decorators)           |
| ------------------ | -------------------------------------- | ----------------------------------- |
| ➕ Add tools        | Define function + register it manually | Just decorate a function            |
| ✏️ Modify tools    | Update function + metadata separately  | Modify function in one place        |
| 🏷️ Organize tools | Manual grouping by name or logic       | Use intuitive **tags**              |
| ❌ Remove tools     | Delete function + unregister it        | Remove the decorator or adjust tags |

---

### 🔁 Understanding the Flow

Here's how the refactored system works under the hood:

1. **Decorator Stage (at load time)**
   All tools decorated with `@register_tool` are added to a **global registry**.

2. **Agent Setup Stage**
   When you create an `ActionRegistry`, it **automatically pulls the tools** matching the specified tags.

3. **Agent Execution Stage**
   The `Agent` uses the selected tools to reason, act, and iterate toward the goal — no need to worry about plumbing!

---

### ✅ Why This Matters

This new structure:

* 🧠 Reduces cognitive load
* 🚫 Prevents metadata drift and broken registrations
* 🧩 Enables modular agent reuse
* ⚡ Boosts velocity when prototyping new agents or tools


